# Data Generation

The purpose of this project is to demonstrate how to regularize models for performance and for fairness. Do do this, we need a method to generate data with bivariate correlations among a number of groups. This is doable with some fun statistics, adapted from the R and SPSS code presented by [David Howell at UVM](https://www.uvm.edu/~statdhtx/StatPages/More_Stuff/CorrGen.html)

The basic steps are
* generate random normal variables for X and y ($\mu=0, \sigma=1$)
* declare a desired correlation coefficient between X and y, $\rho$
* Calculate $a = \rho{\sqrt(1-\rho^2)}$
* Calculate $Z = a*X + y$.
* (optional) Adjust the means and variances of X and Z to what you want them to be by simple linear transformations--(e.g., $ \hat{X} = X^{\prime} * \hat\sigma + \hat\mu$).

We then have to generalize this to a multivariate dataset, which involves the same process but with some matrix algebra, which actually streamlines the process considerably

* create a covariance matrix with desired correlation between all variables (y and X) 
* generate a random multivariate normal distribution with this cov matrix
* adjust means and variances of X and y to fit desired criteria (i.e., de-normalize the data if desired)

## Creating Group Differences

For both cases, the desired outcome is to separate groups within the generated data by a known amount. This is as simple as subsetting the data and adjusting the mean of each group through a linear combination (e.g., $X|g + b$, where b is the effect size of the bias). I've provided examples of three types of common effect sizes for assessing bias:

* Cohen's $d$: standard difference of means between groups. Here, I use the multivariate extention of $d$, which is the standard difference between a single group and the population mean. For example, a data with differences between Male, Female and Non-binary groups might show Female $d = -.07$, indicating that the Female group is 7% of a standard deviation below the mean of all three groups. Effect size $d$ is most applicable for continuous predictions like regression. 
* proportional pass rates, also known as "selection rates": proportional pass rates of each group, independent of each other group. For example, a data with differences between Male, Female and Non-binary groups might show Female $prop = .47$, indicating that the Female group passes the decision threshold 47% of the time. See below for notes on decision thresholds. Most often, proportional pass rates are *dependent*, meaning that they are only indicative of impact relative to other groups. A Female selection rate of .40 is fair if the Male selection rate is .42 ($.40/.42 = .952$), but unfair if the Male selection rate is .60 ($.40/.60 = .667$). This proportion of proportions is known in legal terms as an **impact ratio**. Note that proportional effect size is only applicable to classification. 
* percentile pass rate: the same as proportional pass rates, but based on percentiles rather than proportions.

## Continuous vs Binary Precitions
Both univariate and multivariate instances will generate continuous X and y variables. Converting these variables to binary outcomes is done by boolian logic of $y > threshold$, where the threshold is set to the expected value of the  population mean, 0. 

It is important to note that generated data will *always* approach the corrlations indiated by the covariance matrix given a large enough sample. Binarization of a continuous variable will be proportional to the input covariance matrix, but **will always be** less correlated. This is because binarization of an underlying continuous variable will always increase variance. A better way of correlating binarized continuous variables with a continuous variable might be something like a partial biserial correlation. I'm going to keep that as a future idea for now, though.

## This Notebook

Here, I go through the code inline, and generate samples with different parameters and desired uses.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

# this is for a weird tab completion error on my machine
%config Completer.use_jedi = False


# Univariate

Sometimes you just want two variables to be correlated. This is a fun algebraic method, and pretty easy to set up. 

In [2]:
from typing import Tuple

np.random.seed(8675309)
mu_hat_x = 0
mu_hat_z = 10
sigma_hat_x = 1
sigma_hat_z = 10

def generate_corr_data_univariate(rho: float, 
                                  N: int, 
                                  seed: int=None,
                                  verbose: bool=True, 
                                  reshape_dict: dict=None) -> Tuple[np.ndarray, np.ndarray]:
    """
    Function for simulating univariate correlated data. Given an approximate
    correlation coefficient, rho, simulate data of size N for two random Normal
    variables. Can also include dictionary for reshaping output data.
    
    Parameters
    ----------
    rho : float, range(-1,1)
        value of target approximate correlation for output vectors
        will have normal variance as a function of N
    N : int, range(2,inf)
        length of output
    seed : int
        optional argument to set random seed
    verbose : bool
        argument for printing steps
    reshape_dict : dict[**vargs]
        dictionary to reshape output with the following keys:
        'mu_hat_x' : output mean of X
        'sigma_hat_x' : output standard deviation of X
        'mu_hat_z' : output mean of z
        'sigma_hat_z' : output standard deviation of z        
    
    Returns
    -------
    X, z : np.array
        numpy arrays of size N, approximately correlated at rho
    
    """
    if seed:
        np.random.seed(seed)
    
    X = np.random.normal(0, 1, size=N)
    y = np.random.normal(0, 1, size=N)

    a = rho/np.sqrt(1-rho**2)
    z = a*X+y

    if verbose:
        print(f'generating data with approxmate rho={rho}\n')
        print(f'seeded X mean: {X.mean(): .3f}, sd: {X.std(): .3f}')
        print(f'seeded y mean: {y.mean(): .3f}, sd: {y.std(): .3f}')
        print(f'simulated y (z) mean: {z.mean(): .3f}, sd: {z.std(): .3f}\n')

    # optional, if input moments differ from desired output moments
    if reshape_dict:
        rd = reshape_dict
        X = X*rd['sigma_hat_x']+rd['mu_hat_x']
        z = z*rd['sigma_hat_z']+rd['mu_hat_z']

    if verbose:
        print(f'adjusted X mean: {X.mean(): .3f}, sd: {X.std(): .3f}')
        print(f'adjusted simulated y (z) mean: {z.mean(): .3f}, sd: {z.std(): .3f}\n')
        print(f'correlation of z~X: {pearsonr(X, z)[0]:.3f}')
    
    return z, X

In [3]:
z, X = generate_corr_data_univariate(rho= .5, N = 1000, seed=42)

generating data with approxmate rho=0.5

seeded X mean:  0.019, sd:  0.979
seeded y mean:  0.071, sd:  0.997
simulated y (z) mean:  0.082, sd:  1.126

adjusted X mean:  0.019, sd:  0.979
adjusted simulated y (z) mean:  0.082, sd:  1.126

correlation of z~X: 0.466


In [4]:
z, X = generate_corr_data_univariate(rho= .7, N = 1000, seed=42)

generating data with approxmate rho=0.7

seeded X mean:  0.019, sd:  0.979
seeded y mean:  0.071, sd:  0.997
simulated y (z) mean:  0.090, sd:  1.355

adjusted X mean:  0.019, sd:  0.979
adjusted simulated y (z) mean:  0.090, sd:  1.355

correlation of z~X: 0.678


Reshaping data to have different means and standard deviations, while retaining the approximate correlation we initalized.

In [5]:
reshape_dict={'mu_hat_x' : 0,
        'sigma_hat_x' : 1,
        'mu_hat_z' : 25,
        'sigma_hat_z' : 5}
z, X = generate_corr_data_univariate(rho= .5, N = 1000, seed=1234, reshape_dict=reshape_dict)

generating data with approxmate rho=0.5

seeded X mean:  0.016, sd:  0.973
seeded y mean:  0.042, sd:  0.991
simulated y (z) mean:  0.051, sd:  1.144

adjusted X mean:  0.016, sd:  0.973
adjusted simulated y (z) mean:  25.254, sd:  5.721

correlation of z~X: 0.500


# Multivariate

Say that we want $k$ variables with differing correlations. We can't just repeate the univariate case $k$ times. Rather, we have fun with linearl algebra, and generate a multivariate normal distribution with set covariance between each of the $k$ variables.

Note that not all covariance matrices are possible. All true covariance matrices are constrained to be [positive and semi-definite](https://stats.stackexchange.com/questions/52976/is-a-sample-covariance-matrix-always-symmetric-and-positive-definite), although you can certainly try to coerce an impossible covariance matrix. Scipy will tell you if you try something too weird :)

I've also added some helper functions for generating covariance matrices of a given range. In the more robust `generate_group_data` function, I also include a string labeling convention for small, moderate and large covariances.


In [6]:
from typing import Tuple


def generate_corr_data_multivariate(rmat: np.ndarray, 
                                    N: int, 
                                    seed: int=None,
                                    verbose: bool=True, 
                                    colnames: list=None,
                                    reshape_mu: np.array=None,
                                    reshape_sigma: np.array=None,
                                   ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Function for simulating univariate correlated data. Given an approximate
    correlation coefficient, rho, simulate data of size N for two random Normal
    variables. Can also include dictionary for reshaping output data.
    
    Parameters
    ----------
    rho : float, range(-1,1)
        value of target approximate correlation for output vectors
        will have normal variance as a function of N
    N : int, range(2,inf)
        length of output
    seed : int
        optional argument to set random seed
    verbose : bool
        argument for printing steps
    reshape_mu : np.array
        array to reshape output means
    reshape_sigma : np.array
        array to reshape output standard deviations

    Returns
    -------
    output : pandas.DataFrame
        df shape (N,len(rmat)), with columns approximately correlated at rho
    """
    if seed:
        np.random.seed(seed)
    
    mu = np.zeros(len(rmat))
    
    mat = np.random.multivariate_normal(mu, cov=rmat,size=N)

    # optional, if input moments differ from desired output moments
    if reshape_sigma:
        mat = mat * reshape_sigma
    if reshape_mu:
        mat = mat + reshape_mu
        
    output = pd.DataFrame(mat, columns=colnames)

    if verbose:
        print(f'target rho \n{rmat}')
        print(f'achieved rho \n{pd.DataFrame(mat).corr()}')
    
    return output

In [11]:
from itertools import combinations

def make_cov_array(n, var_range=(0,1)):
    return np.random.uniform(var_range[0], var_range[1], len(list(combinations(range(n),2))))
    

def make_cov_matrix(n, cov_array):
    """
    generate a symetric covariance matrix given an array of
    correlations between columns, in combinatoric order of 
    $_nC_2$
    
    e.g., three element matrix
    cov_array = cor(A,B), cor(A,C), cor(B,C)
    
    e.g., four element matrix
    cov_array = cor(A,B), cor(A,C), cor(A,D), cor(B,C), cor(B,D), cor(C,D)
    """
    M = np.ones([n,n])
    M[np.triu_indices(M.shape[0], k = 1)] = cov_array
    M.T[np.triu_indices(M.shape[0], k = 1)] = cov_array
    return M

def make_random_cov_matrix(n, var_range):
    return make_cov_matrix(n, make_cov_array(n, var_range))
    

In [12]:
make_cov_matrix(3, cov_array=[.1, .2, .3])

array([[1. , 0.1, 0.2],
       [0.1, 1. , 0.3],
       [0.2, 0.3, 1. ]])

In [13]:
make_cov_matrix(4, cov_array=[.1, .2, .3, .4, .5, .6])

array([[1. , 0.1, 0.2, 0.3],
       [0.1, 1. , 0.4, 0.5],
       [0.2, 0.4, 1. , 0.6],
       [0.3, 0.5, 0.6, 1. ]])

In [14]:
make_random_cov_matrix(n=4, var_range=(0,1))

array([[1.        , 0.64739051, 0.39180595, 0.97991933],
       [0.64739051, 1.        , 0.44433563, 0.80767941],
       [0.39180595, 0.44433563, 1.        , 0.96963645],
       [0.97991933, 0.80767941, 0.96963645, 1.        ]])

In [15]:
rmat = make_random_cov_matrix(3, var_range=(0,.4))

cdata = generate_corr_data_multivariate(rmat, 1000, seed=1234, colnames=['y','x1','x2'])
cdata.head()

target rho 
[[1.         0.23341041 0.25925644]
 [0.23341041 1.         0.24453444]
 [0.25925644 0.24453444 1.        ]]
achieved rho 
          0         1         2
0  1.000000  0.233385  0.300786
1  0.233385  1.000000  0.233493
2  0.300786  0.233493  1.000000


,y,x1,x2
0,-1.649692,0.282440,0.372321
1,-0.586274,0.581331,0.667341
2,-0.921669,-0.149230,-0.736351
3,1.622843,0.583204,2.504634
4,-1.472534,0.816746,-1.320984


# Multivariate groupwise correlated data with binary dependent variable

This is what we're really going need, since almost all cases of fairness testing are against categorical variables -- even correlation with age is against age brackets rather than continuous age.

This gets tricky, since we have to mix binomial and multivariate normal distributions. I'm going to cheat, slightly, by converting a multivariate normal distribution to a binary distribution. This has the benefit of approximating the true nature of most classification (as they have some unknown latent continuity under their labels), but has the consequence of only being proportional to our input covariance matrix (see introduction section).

In [18]:
from scipy.stats import percentileofscore

def generate_group_data(n_variables, 
                        groups, 
                        var_cov, 
                        group_ES=None, 
                        es_type='pct',
                        target_name='y', 
                        size=10000,
                        random_seed=None):
        """
        Master function for producing randomly generated datasets with groupwise differences
        for use in testing various fairness systems.
        
        Note that this function will return values of only approximate group differences 
        and covariance. 
        
        Parameters
        ----------
        n_varibles : int
            number of independent variables to generate
            creates one vector for each IV
        groups : list(str)
            list of group names to divide data
            if only one group is provided, generates normally distributed data
        var_cov : np.ndarray or str
            covariance matrix for the desired multivariate normal distribution
            alternatively, can input string commans to generate covariances within range
            'low' : correlations between (0,.2)
            'med' : correlations between (.2,.6)
            'high' : correlations between (.6,1)
        group_ES : list
            effect size metrics for each group, relative to population mean
            see es_type for a description of values and expected behavior
        es_type : str
            type of effect size metrics entered in group_ES
            'pct' and 'prop' are most useful for segmenting groups for classification
            'd' is best used when segmenting groups on continuous metrics
            
            'pct' : percentile pass rates of each group (0, 100)
            'prop': proportional pass rates of each group (0,1)
            'd'   : effect size, d, difference between group and population mean
        target_name : str
            name for dependent variable, aka predictor
            defaults to 'y' by convention
        size : int
            number of rows of data to generate
        random_seed : int
            seed for random state
            
        Returns
        -------
        generated_df : pandas.DataFrame
            dataframe of generated data with covariance, var_cov, 
            and group means segmented by group_ES
        """
        var_names = ['x'+str(i) for i in range(1,n_variables+1)]
        colnames = [target_name] + var_names

        if random_seed:
            np.random.seed(random_seed)

        mu = np.zeros(len(colnames))

        if isinstance(var_cov, str):
            #generate covariance matrix 
            print('generating random covariance matrix\n')
            if var_cov == 'low':
                var_cov = make_random_cov_matrix(n=len(colnames), var_range=(0,.2))
            elif var_cov == 'med':
                var_cov = make_random_cov_matrix(n=len(colnames), var_range=(.2,.6))
            elif var_cov == 'high':
                var_cov = make_random_cov_matrix(n=len(colnames), var_range=(.6,1))

        mat = np.random.multivariate_normal(mu, cov=var_cov,size=size)

        output = pd.DataFrame(mat, columns=colnames)
        output['group'] = None

        batch_size = np.ceil(len(output)/len(groups))

        for b, chunk in output.groupby(np.arange(len(output)) // batch_size):
            b = int(b) #really weird numpy behavior leads to "-0"
            output.loc[chunk.index, 'group'] = groups[b]
            #calculate noncentrality parameter from type of effect size
            if es_type=='prop':
                # calculate from proprortional difference from 0
                prop = group_ES[b]
                # restrict to (0,100)
                pct = max(max(50 + prop*100, 100), 0)
            elif es_type == 'pct':
                # calculate from percentile
                pct = group_ES[b]
            else:
                # calculate from effect size d
                d = group_ES[b]
                # empirical percentile of score
                pct = percentileofscore(chunk.y, chunk.y.mean()+d)
            pct = max(min(pct, 100), 0) #restrict percentile range
            ncp = np.percentile(chunk.y, pct) - chunk.y.mean() 

            output.loc[chunk.index, 'y'] = chunk['y'] + ncp

        output[target_name+'_b'] = np.array(output[target_name] > 0, dtype=int)
        output = output.join(pd.get_dummies(output.group,prefix='group', drop_first=False))
        return output

In [21]:
groups=['M','F']
data = generate_group_data(groups=groups, 
                           n_variables=4, 
                           var_cov='low', 
                           group_ES=[65,50], 
                           es_type='pct',
                           target_name='y', 
                           size=10000)
print(data.groupby('group')['y_b'].mean())

gnames = ['group_'+g for g in groups]
gnames.append('y_b')

data.corr().loc[gnames, gnames]

generating random covariance matrix

group
F    0.4928
M    0.6594
Name: y_b, dtype: float64


,group_M,group_F,y_b
group_M,1.000000,-1.000000,0.168564
group_F,-1.000000,1.000000,-0.168564
y_b,0.168564,-0.168564,1.000000


In [20]:
groups=['Asian','Black','Hispanic/Latino','White']
data = generate_group_data(groups=groups, 
                           n_variables=4, 
                           var_cov='low', 
                           group_ES=[0,-.1,-.15,.1], 
                           es_type='d',
                           target_name='y', 
                           size=10000)
print(data.groupby('group')['y'].mean())

gnames = ['group_'+g for g in groups]
gnames.append('y')

data.corr().loc[gnames, gnames]

generating random covariance matrix

group
Asian              0.001895
Black             -0.106181
Hispanic/Latino   -0.162379
White              0.109364
Name: y, dtype: float64


,group_Asian,group_Black,group_Hispanic/Latino,group_White,y
group_Asian,1.000000,-0.333333,-0.333333,-0.333333,0.023623
group_Black,-0.333333,1.000000,-0.333333,-0.333333,-0.038314
group_Hispanic/Latino,-0.333333,-0.333333,1.000000,-0.333333,-0.070521
group_White,-0.333333,-0.333333,-0.333333,1.000000,0.085212
y,0.023623,-0.038314,-0.070521,0.085212,1.000000


## Demonstrating use of imports

Replicating the above, but with imports rather than inline functions.

In [22]:
import numpy as np
import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%config Completer.use_jedi = False

In [23]:
from fairML.data import data_generator

In [24]:
z, X = data_generator.generate_corr_data_univariate(rho= .5, N = 1000, seed=42)

generating data with approxmate rho=0.5

seeded X mean:  0.019, sd:  0.979
seeded y mean:  0.071, sd:  0.997
simulated y (z) mean:  0.082, sd:  1.126

adjusted X mean:  0.019, sd:  0.979
adjusted simulated y (z) mean:  0.082, sd:  1.126

correlation of z~X: 0.466


In [25]:
rmat = data_generator.make_random_cov_matrix(3, var_range=(0,.4))

cdata = data_generator.generate_corr_data_multivariate(rmat, 1000, seed=1234, colnames=['y','x1','x2'])
cdata.head()

target rho 
[[1.         0.1628426  0.02640394]
 [0.1628426  1.         0.13952821]
 [0.02640394 0.13952821 1.        ]]
achieved rho 
          0         1         2
0  1.000000  0.177243  0.040657
1  0.177243  1.000000  0.152110
2  0.040657  0.152110  1.000000


,y,x1,x2
0,-0.368200,-1.312755,1.197407
1,0.148249,-0.353520,1.052132
2,-0.919647,-0.675096,0.024231
3,2.562133,1.079660,0.720584
4,-2.028659,-0.545323,0.878137


In [26]:
groups=['M','F']
data = data_generator.generate_group_data(groups=groups, 
                           n_variables=4, 
                           var_cov='low', 
                           group_ES=[65,50], 
                           es_type='pct',
                           target_name='y', 
                           size=10000)
print(data.groupby('group')['y_b'].mean())

gnames = ['group_'+g for g in groups]
gnames.append('y_b')

data.corr().loc[gnames, gnames]

generating random covariance matrix

group
F    0.4956
M    0.6564
Name: y_b, dtype: float64


,group_M,group_F,y_b
group_M,1.00000,-1.00000,0.16269
group_F,-1.00000,1.00000,-0.16269
y_b,0.16269,-0.16269,1.00000
